In [1]:
import torch
import torchaudio

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

/root/anaconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
import numpy as np
np_rng = np.random.default_rng(1)
import pandas as pd


import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)


train = pd.read_csv("/tf/datasets/train.tsv", sep="\t")
test = pd.read_csv("/tf/datasets/test.tsv", sep="\t")
dev = pd.read_csv("/tf/datasets/dev.tsv", sep="\t")

train["path"] = train["path"].apply(lambda x: x[:-3] + "mp3")
test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")
dev["path"] = dev["path"].apply(lambda x: x[:-3] + "mp3")

train["split"] = "train"
test["split"] = "test"
dev["split"] = "dev"
vox = pd.read_csv("/tf/datasets/new_dev.tsv", sep="\t")

#test = test.sample(30000, replace=False)


In [3]:
vox = pd.concat([vox] * 4)
vox = vox.iloc[:82449]

In [4]:
vox.shape, train.shape

((82449, 3), (82449, 4))

In [5]:
vox.loc[vox['locale'] == 'ru', 'path'] = vox.loc[vox['locale'] == 'ru', 'path'].apply(lambda x: f"/tf/datasets/vox/ru_dev/{x}")
vox

,Unnamed: 0,path,locale
0,535,common_voice_ta_19093662.mp3,ta
1,147407,common_voice_es_20252168.mp3,es
2,53670,common_voice_it_23740290.mp3,it
3,6394,common_voice_pl_22062475.mp3,pl
4,272713,common_voice_fr_23921769.mp3,fr
...,...,...,...
24394,855,/tf/datasets/vox/ru_dev/6c0Fse9NrvQ__U__S102--...,ru
24395,882,/tf/datasets/vox/ru_dev/n0uowclQ704__U__S136--...,ru
24396,252,/tf/datasets/vox/ru_dev/3QNx2bMb7tM__U__S0---0...,ru
24397,2796,/tf/datasets/vox/ru_dev/CfBlKGX094o__U__S278--...,ru


In [6]:
vox.loc[vox['locale'] == 'en', 'path'] = vox.loc[vox['locale'] == 'en', 'path'].apply(lambda x: f"/tf/datasets/vox/en_dev/{x}")
vox

,Unnamed: 0,path,locale
0,535,common_voice_ta_19093662.mp3,ta
1,147407,common_voice_es_20252168.mp3,es
2,53670,common_voice_it_23740290.mp3,it
3,6394,common_voice_pl_22062475.mp3,pl
4,272713,common_voice_fr_23921769.mp3,fr
...,...,...,...
24394,855,/tf/datasets/vox/ru_dev/6c0Fse9NrvQ__U__S102--...,ru
24395,882,/tf/datasets/vox/ru_dev/n0uowclQ704__U__S136--...,ru
24396,252,/tf/datasets/vox/ru_dev/3QNx2bMb7tM__U__S0---0...,ru
24397,2796,/tf/datasets/vox/ru_dev/CfBlKGX094o__U__S278--...,ru


In [7]:
vox.loc[vox['locale'] == 'kz', 'path'] = vox.loc[vox['locale'] == 'kz', 'path'].apply(lambda x: f"/tf/datasets/vox/kz_dev/{x}")
vox

,Unnamed: 0,path,locale
0,535,common_voice_ta_19093662.mp3,ta
1,147407,common_voice_es_20252168.mp3,es
2,53670,common_voice_it_23740290.mp3,it
3,6394,common_voice_pl_22062475.mp3,pl
4,272713,common_voice_fr_23921769.mp3,fr
...,...,...,...
24394,855,/tf/datasets/vox/ru_dev/6c0Fse9NrvQ__U__S102--...,ru
24395,882,/tf/datasets/vox/ru_dev/n0uowclQ704__U__S136--...,ru
24396,252,/tf/datasets/vox/ru_dev/3QNx2bMb7tM__U__S0---0...,ru
24397,2796,/tf/datasets/vox/ru_dev/CfBlKGX094o__U__S278--...,ru


In [8]:
vox.loc[vox['locale'] == 'kz']

,Unnamed: 0,path,locale
15966,1021,/tf/datasets/vox/kz_dev/4QuKKCY5mAo__U__S151--...,kz
15967,3210,/tf/datasets/vox/kz_dev/YNTYeSoy0HM__U__S285--...,kz
15968,1849,/tf/datasets/vox/kz_dev/Gu-TfzMwS6g__U__S271--...,kz
15969,108,/tf/datasets/vox/kz_dev/ASsJmutpNjg__U__S19---...,kz
15970,49,/tf/datasets/vox/kz_dev/O0YlWuOb-YQ__U__S16---...,kz
...,...,...,...
23083,356,/tf/datasets/vox/kz_dev/23UJ403sOxA__U__S128--...,kz
23084,545,/tf/datasets/vox/kz_dev/c2VaP6HXWS0__U__S41---...,kz
23085,250,/tf/datasets/vox/kz_dev/xg8wrC8Gao4__U__S119--...,kz
23086,2330,/tf/datasets/vox/kz_dev/HIPlXLT3aYw__U__S149--...,kz


In [9]:
vox.loc[(vox['locale'] != "kz") & (vox['locale'] != "ru") & (vox['locale']  != "en"), "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" + vox.loc[(vox['locale'] != "kz") & (vox['locale'] != "ru") & (vox['locale']  != "en")]["locale"]  + "/clips/" + vox.loc[(vox['locale'] != "kz") & (vox['locale'] != "ru") & (vox['locale']  != "en")]["path"]

2021-07-07 03:05:32.049 I numexpr.utils: Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2021-07-07 03:05:32.050 I numexpr.utils: Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-07-07 03:05:32.051 I numexpr.utils: NumExpr defaulting to 8 threads.


In [10]:
vox = vox.reset_index()

In [11]:
train["target_domain"] = vox['path']

In [12]:
meta = pd.concat([train, test, dev])


In [13]:
meta

,Unnamed: 0,path,locale,split,target_domain
0,1486,common_voice_ru_19559882.mp3,ru,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
1,56701,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
2,3364,common_voice_ru_20957735.mp3,ru,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
3,110475,common_voice_rw_22681269.mp3,rw,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
4,45384,common_voice_en_20767496.mp3,en,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
...,...,...,...,...,...
30751,2677,common_voice_ru_18947939.mp3,ru,dev,NaN
30752,881,common_voice_ru_18853495.mp3,ru,dev,NaN
30753,68709,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,dev,NaN
30754,249025,common_voice_en_21725213.mp3,en,dev,NaN


In [14]:
meta.loc[meta["locale"] != "kz", "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" +  meta.loc[meta["locale"] != "kz"]["locale"] + "/clips/" + meta.loc[meta["locale"] != "kz"]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = meta["Unnamed: 0"].apply(str)
meta["target"] = meta["target"].map(targets)

meta

,Unnamed: 0,path,locale,split,target_domain,target,id
0,1486,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,1,1486
1,56701,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,0,56701
2,3364,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,1,3364
3,110475,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,rw,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,3,110475
4,45384,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,en,train,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,2,45384
...,...,...,...,...,...,...,...
30751,2677,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,dev,NaN,1,2677
30752,881,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,dev,NaN,1,881
30753,68709,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,dev,NaN,0,68709
30754,249025,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,en,dev,NaN,2,249025


In [15]:
from torch.utils.data import Dataset
import random
import math
DEVICE = 'cuda'
def _get_sample(path, resample=None):
  effects = [
    ["remix", "1"]
  ]
  if resample:
    effects.append(["rate", f'{resample}'])
  return torchaudio.sox_effects.apply_effects_file(path, effects=effects)

SAMPLE_RIR_PATH = os.path.join(os.getcwd(), "rir.wav")

def get_rir_sample(*, resample=None, processed=False):
    rir_raw, sample_rate = _get_sample(SAMPLE_RIR_PATH, resample=resample)
    if not processed:
        return rir_raw, sample_rate
    rir = rir_raw[:, int(sample_rate*1.01):int(sample_rate*1.3)]
    rir = rir / torch.norm(rir, p=2)
    rir = torch.flip(rir, [1])
    return rir, sample_rate

class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, tg=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        self.target_domain = tg
        self.rir = get_rir_sample()[0]
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx], normalization=True)
        
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(y)
        y = torchaudio.transforms.Vad(sample_rate = 16000)(y)

     
        fixed_length = 16000 * 12
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        representation = y
        
        
        if self.target_domain is None:
            y = []
        else:
            y, sr = torchaudio.load(self.target_domain.iloc[idx], normalization=True)


            y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(y)
            y = torchaudio.transforms.Vad(sample_rate = 16000)(y)


            fixed_length = 16000 * 12

            # returning result
            if y.shape[1] < fixed_length:
                y = torch.nn.functional.pad(
                  y, (0, fixed_length - y.shape[1]))
            else:
                y = y[:, :fixed_length]
        
        result = {"target":self.targets.iloc[idx], "representation":representation, "target_domain": y}

        return result

In [16]:
ds = AudiosDataset(meta["path"], meta["target"], meta["target_domain"])

In [17]:
from torch.utils.data import TensorDataset, DataLoader
train_ds = AudiosDataset(meta.loc[meta["split"]=="train"]["path"], meta.loc[meta["split"]=="train"]["target"],  meta.loc[meta["split"]=="train"]["target_domain"], augment=True)
val_ds = AudiosDataset(meta.loc[meta["split"]=="dev"]["path"], meta.loc[meta["split"]=="dev"]["target"])
test_ds = AudiosDataset(meta.loc[meta["split"]=="test"]["path"], meta.loc[meta["split"]=="test"]["target"])

In [18]:
batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "valid": DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test":DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [19]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-07-07 03:05:50.991 I filelock: Lock 139785345586768 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www

/root/anaconda3/lib/python3.7/site-packages/torchaudio/functional.py:318: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/root/anaconda3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/root/anaconda3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Trigger

In [20]:
from torch import nn
import torch.nn.functional as F
class Extractor(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor

        self.rnn = nn.LSTM(input_size=768, hidden_size=512, num_layers=5, bidirectional=True, batch_first=True, dropout=0.7)
        
    def forward(self, x):
        features = self.extractor(torch.squeeze(x))
        features = torch.stack(features)
        res, _ = self.rnn(features)
        return res[:, -1, :]
    
    
    
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.classifier = nn.Sequential(
                    nn.Linear(1024, 256),
                    nn.ReLU(),
                    nn.BatchNorm1d(256),
                    nn.Dropout(0.5),
                    nn.Linear(256, 4),
                    )
        
    def forward(self, x):
        
        res = self.classifier(x)
        return F.log_softmax(res, dim=1)

In [21]:
G = Extractor(m)
C1 = Classifier()
C2 = Classifier()

In [26]:
G.load_state_dict(torch.load("extractor.h5"))
C1.load_state_dict(torch.load("clf1.h5"))
C2.load_state_dict(torch.load("clf2.h5"))

<All keys matched successfully>

In [22]:
import torch.optim as optim
opt_g = optim.Adam(G.parameters(),
                                    lr=5e-5, weight_decay=0.0005)

opt_c1 = optim.Adam(C1.parameters(),
                                     lr=5e-5, weight_decay=0.0005)
opt_c2 = optim.Adam(C2.parameters(),
                                     lr=5e-5, weight_decay=0.0005)

In [23]:
from tqdm import tqdm


def discrepancy(out1, out2):
        return torch.mean(torch.abs((out1) - (out2)))

def reset_grad():
    opt_g.zero_grad()
    opt_c1.zero_grad()
    opt_c2.zero_grad()
    
    
    

def train(criterion, epochs, data_tr, data_val, max_stable=5, num_K = 1):
    best_val_loss = 1e9
    counter = 0
    
    for epoch in range(epochs):
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        G.train()
        C1.train()
        C2.train()  # train mode
        for batch in tqdm(data_tr):
            loss = 0
            # data to device
            X_batch, Y_batch, targ = batch["representation"], batch["target"], batch["target_domain"]
            #print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            targ = targ.to(DEVICE)
            # set parameter gradients to zero
            reset_grad()
            # forward
            features = G(X_batch)
            output_s1 = C1(features)
            output_s2 = C2(features)

            loss_s1 = criterion(output_s1, Y_batch)
            loss_s2 = criterion(output_s2, Y_batch)
            loss_s = loss_s1 + loss_s2
            loss_s.backward()
            opt_g.step()
            opt_c1.step()
            opt_c2.step()
            reset_grad()
            
            
            feat_s = G(X_batch)
            output_s1 = C1(feat_s)
            output_s2 = C2(feat_s)
            feat_t = G(targ)
            output_t1 = C1(feat_t)
            output_t2 = C2(feat_t)

            loss_s1 = criterion(output_s1, Y_batch)
            loss_s2 = criterion(output_s2, Y_batch)
            loss_s = loss_s1 + loss_s2
            loss_dis = discrepancy(output_t1, output_t2)
            loss = loss_s - loss_dis
            loss.backward()
            opt_c1.step()
            opt_c2.step()
            reset_grad()
            for i in range(num_K):
                #
                feat_t = G(targ)
                output_t1 = C1(feat_t)
                output_t2 = C2(feat_t)
                loss_dis = discrepancy(output_t1, output_t2)
                loss_dis.backward()
                opt_g.step()
                reset_grad()
        # show intermediate results
        G.eval()
        C1.eval()
        C2.eval() 
        val_loss = 0
        print("start validation")
        for v_b in tqdm(data_val):
            X_val, Y_val = v_b["representation"], v_b["target"]
            feat = G(X_val.to(DEVICE))
            pred_1 = C1(feat).detach().cpu()
            pred_2 = C2(feat).detach().cpu()
            val_loss += (criterion(pred_1, Y_val) + criterion(pred_2, Y_val)) / 2
        val_loss /= len(data_val)
        print( f"validation loss: {val_loss}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            print("Save new model!")
            best_val_loss = val_loss
            torch.save(G.state_dict(), f'extractor.h5')
            torch.save(C1.state_dict(), f'clf1.h5')
            torch.save(C2.state_dict(), f'clf2.h5')

        else:
            counter += 1
        if counter == max_stable:
            break

In [24]:
DEVICE = 'cuda'
max_epochs = 100
G = G.to(DEVICE)
C1 = C1.to(DEVICE)
C2 = C2.to(DEVICE)


In [25]:
loss_fn =  nn.CrossEntropyLoss()
train(loss_fn, max_epochs, loaders["train"], loaders["valid"])

  0%|          | 0/10306 [00:00<?, ?it/s]

* Epoch 1/100


  0%|          | 2/10306 [01:21<116:26:52, 40.68s/it]


KeyboardInterrupt: 

In [30]:
true_labels = []
predicted_labels = []
for batch in tqdm(loaders["test"]):
    G.eval()
    C1.eval()
    prediction = G(batch["representation"].to(DEVICE))
    prediction = C1(prediction).detach().cpu()
    predicted_labels.extend(torch.argmax(prediction, dim=1).tolist())
    true_labels.extend(batch["target"].tolist())


100%|██████████| 6971/6971 [31:41<00:00,  3.67it/s]  


In [31]:
from sklearn.metrics import classification_report

report = classification_report(true_labels, predicted_labels, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       1.00      0.99      0.99     17341
          ru       0.43      0.00      0.00     10379
          en       0.00      0.00      0.00     12964
       other       0.39      1.00      0.56     15084

    accuracy                           0.58     55768
   macro avg       0.45      0.50      0.39     55768
weighted avg       0.50      0.58      0.46     55768



/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
import numpy as np
np_rng = np.random.default_rng(1)


import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)


test = pd.read_csv("/tf/datasets/new_test.tsv", sep="\t")

test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")

test["split"] = "test"
meta = pd.concat([test])


In [33]:
G

Extractor(
  (extractor): UpstreamExpert(
    (transformer): PretrainedTransformer(
      (extracter): OnlinePreprocessor(
        (_melscale): MelScale()
        (_mfcc_trans): MFCC(
          (amplitude_to_DB): AmplitudeToDB()
          (MelSpectrogram): MelSpectrogram(
            (spectrogram): Spectrogram()
            (mel_scale): MelScale()
          )
        )
      )
      (model): TransformerModel(
        (input_representations): TransformerInputRepresentations(
          (spec_transform): Linear(in_features=80, out_features=768, bias=True)
          (LayerNorm): TransformerLayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): TransformerEncoder(
          (layer): ModuleList(
            (0): TransformerLayer(
              (attention): TransformerAttention(
                (self): TransformerSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, o

In [34]:
meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en")))), "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["locale"]  + "/clips/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = meta["Unnamed: 0"].apply(str)
meta["target"] = meta["target"].map(targets)


workdir = "/tf/datasets/transformer"


In [35]:
meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru")]["path"].apply(lambda x: f"/tf/datasets/vox/ru_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"]
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz")]["path"].apply(lambda x: f"/tf/datasets/vox/kz_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] 
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "en")]["path"].apply(lambda x: f"/tf/datasets/vox/en_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] 

0       /tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
1       /tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
2       /tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
3       /tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
4       /tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
                              ...                        
9995    /tf/datasets/vox/en_test/KLiy94kfZI4__U__S133-...
9996    /tf/datasets/vox/en_test/YTlliEr5LOA__U__S113-...
9997    /tf/datasets/vox/en_test/bSs0gNq6Kkc__U__S0---...
9998    /tf/datasets/vox/en_test/Da7c-BY6MDA__U__S2---...
9999    /tf/datasets/vox/en_test/VWvPndMo1F8__U__S24--...
Name: path, Length: 10000, dtype: object

In [36]:
meta.loc[meta["split"]=="test", "Unnamed: 0"] = meta.loc[meta["split"]=="test"]["path"]

In [37]:
meta["id"] = meta["Unnamed: 0"].apply(str)

In [38]:
meta.loc[meta["split"] == "test", "id"] = meta.loc[meta["split"] == "test"]["path"]

In [39]:
meta = meta.set_index("Unnamed: 0")
meta.loc[meta["split"]=="test"]

,path,locale,split,target,id
Unnamed: 0,,,,,
/tf/datasets/vox/en_test/shrDRhToGpY__U__S133---0944.430-0958.260.mp3,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,en,test,2,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123---0427.020-0444.670.mp3,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...,en,test,2,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---0398.760-0403.940.mp3,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...,en,test,2,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---1473.480-1485.720.mp3,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...,en,test,2,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---0125.230-0140.900.mp3,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...,en,test,2,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
...,...,...,...,...,...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/it/clips/common_voice_it_20015623.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,it,test,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/uk/clips/common_voice_uk_23554602.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,uk,test,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_20416266.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,tr,test,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...


In [40]:
meta.loc[meta["split"] == "test"] = meta.loc[(meta["split"] == "test") & (meta["target"] != 3)] 

In [41]:
meta.loc[meta["split"]=="test", "id"] = meta.loc[meta["split"]=="test"]["path"]
meta.loc[meta["split"]=="test"] 

,path,locale,split,target,id
Unnamed: 0,,,,,
/tf/datasets/vox/en_test/shrDRhToGpY__U__S133---0944.430-0958.260.mp3,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,en,test,2.0,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123---0427.020-0444.670.mp3,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...,en,test,2.0,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---0398.760-0403.940.mp3,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...,en,test,2.0,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---1473.480-1485.720.mp3,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...,en,test,2.0,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---0125.230-0140.900.mp3,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...,en,test,2.0,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
...,...,...,...,...,...
/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25---0107.830-0127.780.mp3,/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25--...,kz,test,0.0,/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25--...
/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26---0236.830-0241.550.mp3,/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26--...,kz,test,0.0,/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26--...
/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78---0466.720-0470.860.mp3,/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78--...,kz,test,0.0,/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78--...


In [42]:
newtest_ds = AudiosDataset(meta.loc[meta["split"]=="test"]["path"], meta.loc[meta["split"]=="test"]["target"])

In [44]:
loaders["new_test"] = DataLoader(
        newtest_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    )
DEVICE = 'cuda'


In [45]:
true_labels = []
predicted_labels = []
for batch in tqdm(loaders["new_test"]):
    G.eval()
    C1.eval()
    prediction = G(batch["representation"].to(DEVICE))
    prediction = C1(prediction).detach().cpu()
    predicted_labels.extend(torch.argmax(prediction, dim=1).tolist())
    true_labels.extend(batch["target"].tolist())



100%|██████████| 4506/4506 [17:31<00:00,  4.29it/s]  


In [46]:
from sklearn.metrics import classification_report

report = classification_report(true_labels, predicted_labels, target_names=list(targets.keys()), labels=range(4))
print(report)



/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          kz       0.38      0.86      0.52     13941
          ru       0.40      0.00      0.00     12107
          en       0.00      0.00      0.00     10000
       other       0.00      0.00      0.00         0

    accuracy                           0.33     36048
   macro avg       0.19      0.21      0.13     36048
weighted avg       0.28      0.33      0.20     36048

